# What you need to know first.

## Functors

#### What is a functor?
A functor is a class who's objects are meant to be called like a function.

#### Why not just functions?
Functions are great! Simple and effective. But having a entire library of functions with oddly specific names that require you to put in what pin you want to perfom the action every time gets tiresome and is hard to read. Also, if you change how a pin works you might have to change it a throughout your code.

#### Why not just classes?
Classes are good but end up getting complex and messy once the code gets too big. This gets extra bad in python because you might want to access members as well as functions for this or that.

In [3]:
class PersonMood:
    def __init__(self, name):
        self.name = name
        
    def __call__(self, mood):
        print(f'{self.name} is now {mood}')


functor = PersonMood('Amber')
functor('sad')
functor('happy')

Amber is now sad
Amber is now happy


## Lambda (anonomous) functions
A lambda function is simply an inline function.
It returns the result of the right hand side of the colon, while the left is used to define the arguments

In [ ]:
lambda_function = lambda arg1, arg2: arg1+arg2
lambda_function(4, 6)

## Single Pin Data
Machines are controlled by simple eletrical signals many of these can be done with a single pin.

#### Digital
The pin can have voltage or not. This is quite effective for controlling transistors for providing power to subsystems.

#### Analog
The pin's voltage might be variable. To ajust speed or to indicate a battery level.

#### PWM
Pulse Width Modulation. This is what servo motors and linear actuators are controlled with. It also has many other uses. Duty-cycle is a simulated analog voltage by applying the full voltage a certain percentage of the time.
Pulse Width Modulation is very broad and covers a large range of uses. PPM or Pulse Positon Modulation is the standard range of PWM for servos.


## Multi-pin Data
This is a work in progress. For my library it is not yet supported.

#### UART

#### I2C

#### SPI

#### USB

# How MachineIO works
MachineIO is a tool to help you create your own function library for interfacing with your hardware, exactly the way you want and nothing more.

### Device
You first create a device. Devices work with protocols MachineIO handles everything to translate your libarary to the proper device in the background. The device is a traditional object used simply as a parameter for the functors.


In [ ]:
from machineio import *
test_device = Device('test')
print(test_device)

### Pin
This is the heart of machineio defining a pin is simple and readable.
all pins need to have a device they are on, so it's the first argument.
The second argument is the pin number, the hardware will have documention on what pin is what number.
a Pin is an INPUT or an OUTPUT, these are flags in machineio for readability.
a Pin type flags are given to specify the mode/role any modifications required will be passed in as keywords.
the halt keyword argument is the stopped/safe of the pin.

Pins have a lot more functionality, if you would like more see the docs.

In [ ]:
pin_functor = Pin(test_device, 1, OUTPUT, Servo(),
                  halt=lambda self: self(0))

input_pin = Pin(test_device, 2, INPUT, Analog(),
                callback=lambda: print('Callback function'))

advanced_pin = Pin(test_device, 3, OUTPUT, PWM(),
                   halt=lambda self: self(0),
                   limits=(0, 100),
                   translate=lambda x: x*1.8)

# Group
You hardly ever have one pin that does everything you need to perform an action.
This is where groups excell. They allow you to easly create groups of pins.
Groups can be multi-dimentional. If they are, add transltions to isolate or modify for pins calls.
You can add delays to certain pins in a group if some need to act before others. (This is an async fire and forget call)

In [ ]:
x = Pin(test_device, 4, OUTPUT, Servo(), halt=lambda self: self(0))
x = Pin(test_device, 5, OUTPUT, Servo(), halt=lambda self: self(0))

planar = Group(2)

planar.add(x, translate=lambda x, y: x)
planar.add(y, translate=lambda x, y: y, delay=1.2)

planar(2, 8)

# Safety
Groups and pins have halt keywords that use lambda functions to call themselfs with the pin or group as its own parameter to call it like so `lambda self: self(0)`
when you create a pin or group it regsters itself to safety.
If you call the machineio.kill() it will stop every Pin and totally ignore groups.
If you call machineio.stop() it will stop all groups with a halt, and all pins not in a group.
If you have nested groups the behavior is well defined but more complex, see documention.
Any calls after this event will throw an exception until you reset the event with machineio.safety.proceed = True

In [ ]:
mio.kill()
mio.safety.proceed = True
mio.stop()
mio.safety.proceed = True